# Libraries and more

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install opencv-python
#!pip install matplotlib
#!pip install numpy
!pip install scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import math
import numpy as np
import pywt
import os
import copy
import pandas as pd

from skimage.feature import local_binary_pattern
from skimage import measure, data, feature
from scipy.stats import kurtosis, skew
from math import sqrt
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
from fastprogress import master_bar, progress_bar

In [ ]:
# variables

DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')

_, _, images = next(os.walk(os.path.join(DATA_DIR,'images')))
_, _, groundTruths = next(os.walk(os.path.join(DATA_DIR, 'groundtruths')))

images.sort()
groundTruths.sort()

# read numbers of normal images
normals = []
with open(os.path.join(DATA_DIR,'normals.txt')) as f:
    for line in f:
        normals.append(line[:-1])

# Functions for Image Processing

## Preprocessing

For preprocessing we have several steps options:


1.   Dehazing with Dark Channel Prior and Guided Filter
2.   CLAHE (contrast limited adaptive histogram equalization)



### DeHazing Using Dark Channel Prior and Guided Filter

In [ ]:
# Here goes inputs --> output types
def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

In [ ]:
def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx;
    return A

In [ ]:
def TransmissionEstimate(im,A,sz):
    omega = 0.95
    im3 = np.empty(im.shape,im.dtype)

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz)
    return transmission

In [ ]:
def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

In [ ]:
def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);
    return t;

In [ ]:
def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);
    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]
    return res

In [ ]:
def deHazingDarkChannelPrior(matrix):

  I = matrix.astype(np.float64)/255

  dark = DarkChannel(I,15)
  A = AtmLight(I,dark)
  te = TransmissionEstimate(I,A,15)
  t = TransmissionRefine(matrix,te)
  J = Recover(I,t,A,0.1)
  preprocessed = J
  return preprocessed

### CLAHE

In [ ]:
def imgCLAHE(matrix):
  matrix = matrix.astype(np.uint16)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  cl1 = clahe.apply(matrix)
  return cl1

## Candidate Extraction

### Determinant of Hessian

In [ ]:
def candidateExtractionDoH(matrix):
  
  # returns x,y,sigma of the blob
  blobs = feature.blob_doh(matrix,
                           min_sigma=1,
                           max_sigma=30,
                           num_sigma=10,
                           threshold=0.005,
                           # lower more sensible, more false positives bad also tinier calcifications detected
                           overlap=0.5,
                           log_scale=False,
                           threshold_rel=None
                           )
  # taken from the documentation
  # ...The downside is that this method can’t be used for detecting blobs of radius less than 3px
  # due to the box filters used in the approximation of Hessian Determinant.
  result = blobs
  return result

### Difference of Gaussians

In difference of gaussians for candidate extraction we have different configuration of parameters

In [ ]:
# studied options:
# option 1:
# min_sigma=0.0001
# max_sigma=30,
# threshold=0.04

# option 2:
# min_sigma=0.005
# max_sigma=50,
# threshold=0.04

# default option 1
def candidateExtractionDoG(matrix, minSigma=0.0001, maxSigma=30, threshold=0.04):

  blobs = feature.blob_dog(matrix,
                           min_sigma=minSigma,
                           max_sigma=maxSigma,
                           threshold=threshold)

  # https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_blob.html
  blobs[:, 2] = blobs[:, 2] * sqrt(2) 

  result = blobs
  return result

## Feature Extraction

### Labelling patches

In [ ]:
# Get Ground Truth for each patch
def patchGroundTruth(top, bottom, left, right, groundTruth):
  result = str(0)
  truePatch = groundTruth[top : bottom,
                          left : right]
  
  sum = np.sum(truePatch)

  if sum > 0:
    result = str(1)
  
  return result

### First Order Statistic Features

In [ ]:
def functionFeatures(patch):
  import scipy
  # 2) Parameters
  roi = patch.ravel()
  level_min = 0
  level_max = max(roi)
  Ng = (level_max - level_min) + 1
  bins = Ng
  
  # 3) Calculate Histogram H inside ROI
  H = np.histogram(roi, bins=bins, range=[level_min, level_max], density=True)[0]

  # 4) Calculate Features
  fos = {}
  i = np.arange(0,bins)
  # 0 - mean
  fos["Mean"] = np.mean(roi)
  # 1 - variance
  fos["Variance"] = np.var(roi)
  # 2 - median
  fos["Median"] = np.median(roi) 
  # 3 - mode
  fos["Mode"] = scipy.stats.mode(roi)[0][0]
  # 4 - skewness
  fos['skewness'] = skew(roi)
  # 5 - kurtosis
  fos['kurtosis'] = kurtosis(roi)
  # 6 - energy
  fos["Energy"] = sum(np.multiply(H,H))
  # 7 - entropy
  fos["Entropy"] = -sum(np.multiply(H,np.log(H+1e-16)))##################
  # 8 - minimal gray level
  fos["MinimalGrayLevel"] = min(roi)
  # 9 - maximal gray level
  fos["MaximalGrayLevel"] = max(roi)
  # 10 - coefficient of variation
  fos["CoefficientOfVariation"] = np.sqrt(fos["Median"]) / fos["Mean"]
  # 11 - 10 percentile
  fos["10Percentile"] = np.percentile(roi,10) 
  # 12 - 25 percentile
  fos["25Percentile"] = np.percentile(roi,25)  
  # 13 - 75 percentile
  fos["75Percentile"] = np.percentile(roi,75) 
  # 14 - 90 percentile
  fos["90Percentile"] = np.percentile(roi,90) 
  # 15 - histogram width
  fos["HistogramWidth"] = fos["90Percentile"] - fos["10Percentile"]

  return fos

### More Features for the Co-ocurrency matrix

In [ ]:
def ProbSumDiff(glcm):
  resultSum = np.zeros(glcm.shape[0] * 2)
  resultDiff = np.zeros(glcm.shape[0] * 2)
  for i in range(0, glcm.shape[0]):
    for j in range(0, glcm.shape[1]):
      resultSum[i + j] += glcm[i,j]
      resultDiff[abs(i - j)] += glcm[i,j]
  return resultSum, resultDiff

def glcmFeatures(glcm):
  fglcm = {}

  #epsilon
  eps = 0.00000001

  # sum probabilities
  probSum, probDiff = ProbSumDiff(glcm)
  
  sumEntropy = 0.0
  diffEntropy = 0.0

  for i in range(0, len(probSum)):
    # sum of entropy
    sumEntropy += probSum[i] * np.log(probSum[i] + eps)
    # diff of entropy
    diffEntropy += probDiff[i] * np.log(probDiff[i] + eps)

  fglcm['sumEntropy'] = -1 * sumEntropy
  fglcm['diffEntropy'] =  -1 * diffEntropy
  
  sumVariance = 0.0
  sumAverage = 0.0
  diffVariance = 0.0

  for i in range(0, len(probSum)):
    # sum of variance
    sumVariance += (i - sumEntropy) * (i - sumEntropy) * probSum[i]
    # sum of average
    sumAverage += i * probSum[i]
    # diff of variance
    diffVariance += (i - diffEntropy) * (i - diffEntropy) * probDiff[i]

  fglcm['sumVariance'] = sumVariance
  fglcm['sumAverage'] = sumAverage
  fglcm['diffVariance'] = diffVariance
  
  return fglcm

### Save features file

In [ ]:
def writeFeatures(features, flag, folder, image, name):
  if(flag):
    features.to_csv(os.path.join(folder,
                                 name + '.csv'),
                    mode='a',
                    index=False)
    flag = False
  else:
    features.to_csv(os.path.join(folder,
                                 name + '.csv'),
                  mode='a',
                  header=False,
                  index=False)
  return flag

### Main feature functions

Haar + GLCM

In [ ]:
from pandas._libs.index import IndexEngine
from pandas.core.indexing import IndexingError

def featuresExtractionHaarGLCM(matrix, candidates, features, image, folder):
  
  # no candidates, no extraction needed
  if (len(candidates) == 0):
    return []

  # file
  flag = True

  # for each candidate
  for index, candidate in enumerate(progress_bar(candidates)):

    # to use them as coordinates they have to be integers
    candidate = candidate.astype(np.int64)

    # candidates are y,x and sigma
    # tolerance to the window
    n = 7

    # if it is not possible a square patch
    if (((candidate[1] - n) < 0) or 
        ((candidate[1] + n) > matrix.shape[0]) or 
        ((candidate[0] - n) < 0) or
        ((candidate[0] + n) > matrix.shape[1])):
      # ignore it
      continue

    # defining limits 
    left = int(candidate[1] - n)
    right = int(candidate[1] + n)
    top = int(candidate[0] - n)
    bottom = int(candidate[0] + n)

    # getting patch / roi
    patchCandidate = matrix[top : bottom,
                            left : right]

    # getting haar n= 2
    n = 2
    w = 'haar'
    coeffs = pywt.wavedec2(patchCandidate,wavelet=w,level=n)
    LL2, (LH2, HL2, HH2), (LH1, HL1, HH1) = coeffs
    
    # # decompose haar patches to use for feature extraction
    dictHaar = {'LL2':LL2, 'LH2':LH2, 'HL2': HL2, 'HH2': HH2, 'LH1':LH1, 'HL1': HL1, 'HH1': HH1}

    # # for each haar decomposition
    for key, haar in dictHaar.items():

      # starting the row values
      dictFeatures = {}
      dictFeatures = {'name': 'patch_' + str(index) + '_' + str(key) + '_' + str(image.split(".")[0]),
                      #'label': patchGroundTruth(top, bottom, left, right, groundTruth),
                      'x': candidate[0],
                      'y': candidate[1]}

      # compute first order statistic features
      dictFos = functionFeatures(haar.astype(np.uint8))
      # saving it in the row values
      dictFeatures.update(dictFos)

      # Relative smoothness
      dictFeatures['relativeSmoothness'] = 1 - ( 1 / (1.0 + dictFeatures["Variance"]))
      

      # input image, distance in pixels, angles
      glcm = feature.greycomatrix(haar.astype(np.uint8), [ 1 ], [ 0 ])
      
      # glcm features    
      # properties
      dictFeatures['contrast'] = feature.greycoprops(glcm, 'contrast')[0][0]
      dictFeatures['dissimilarity'] = feature.greycoprops(glcm, 'dissimilarity')[0][0]
      dictFeatures['homogeneity'] = feature.greycoprops(glcm, 'homogeneity')[0][0]
      dictFeatures['energy'] = feature.greycoprops(glcm, 'energy')[0][0]
      dictFeatures['correlation'] = feature.greycoprops(glcm, 'correlation')[0][0]
      dictFeatures['ASM'] = feature.greycoprops(glcm, 'ASM')[0][0]

      dictFeatures['entropy'] = measure.shannon_entropy(glcm)

      fglcm = glcmFeatures(glcm[:,:,0,0])
      dictFeatures.update(fglcm)

      del glcm

      # add to the dataframe the features for this patch
      features = features.append(dictFeatures, ignore_index=True)

  # save in the csv
  writeFeatures(features, flag, folder, image, '_haar_glcm')

  return features

LBP+GLCM

In [ ]:
def featuresExtractionLBPGLCM(matrix, candidates, features, image, folder):
  
  # no candidates, no extraction needed
  if (len(candidates) == 0):
    return []

  # file
  flag = True

  # for each candidate
  for index, candidate in enumerate(progress_bar(candidates)):

    # to use them as coordinates they have to be integers
    candidate = candidate.astype(np.int64)

    # tolerance to the window
    n = 7

    # if it is not possible a square patch
    if (((candidate[1] - n) < 0) or 
        ((candidate[1] + n) > matrix.shape[0]) or 
        ((candidate[0] - n) < 0) or
        ((candidate[0] + n) > matrix.shape[1])):
      # ignore it
      continue

    # defining limits 
    left = int(candidate[1] - n)
    right = int(candidate[1] + n)
    top = int(candidate[0] - n)
    bottom = int(candidate[0] + n)

    # getting patch / roi
    patchCandidate = matrix[top : bottom,
                            left : right]

    # LBP received grayscale

    radius= 1
    points= 8 * radius

    patchCandidate = local_binary_pattern(patchCandidate, points, radius, method='default')

    # starting the row values
    dictFeatures = {}
    dictFeatures = {'name': 'patch_' + str(index) + '_' + str(image.split(".")[0]),
                    #'label': patchGroundTruth(top, bottom, left, right, groundTruth),
                    'x': candidate[0],
                    'y': candidate[1]}

    # compute first order statistic features
    dictFos = functionFeatures(patchCandidate.astype(np.uint8))
    
    # saving it in the row values
    dictFeatures.update(dictFos)

    # Relative smoothness
    dictFeatures['relativeSmoothness'] = 1 - ( 1 / (1.0 + dictFeatures["Variance"]))
    

    # input image, distance in pixels, angles
    glcm = feature.greycomatrix(patchCandidate.astype(np.uint8), [ 1 ], [ 0 ])
    
    # glcm features    
    # properties
    dictFeatures['contrast'] = feature.greycoprops(glcm, 'contrast')[0][0]
    dictFeatures['dissimilarity'] = feature.greycoprops(glcm, 'dissimilarity')[0][0]
    dictFeatures['homogeneity'] = feature.greycoprops(glcm, 'homogeneity')[0][0]
    dictFeatures['energy'] = feature.greycoprops(glcm, 'energy')[0][0]
    dictFeatures['correlation'] = feature.greycoprops(glcm, 'correlation')[0][0]
    dictFeatures['ASM'] = feature.greycoprops(glcm, 'ASM')[0][0]

    dictFeatures['entropy'] = measure.shannon_entropy(glcm)
    
    fglcm = glcmFeatures(glcm[:,:,0,0])
    dictFeatures.update(fglcm)

    del glcm

    # add to the dataframe the features for this patch
    features = features.append(dictFeatures, ignore_index=True)

  # save in the csv
  writeFeatures(features, flag, folder, image, '_lbp_glcm')

  return features

GLCM

In [ ]:
from skimage import feature

def featuresExtractionGLCM(matrix, candidates, features, image, folder):
  
  # no candidates, no extraction needed
  if (len(candidates) == 0):
    return []

  # file
  flag = True

  # for each candidate
  for index, candidate in enumerate(progress_bar(candidates)):

    # to use them as coordinates they have to be integers
    candidate = candidate.astype(np.int64)

    # tolerance to the window
    n = 7

    # if it is not possible a square patch
    if (((candidate[1] - n) < 0) or 
        ((candidate[1] + n) > matrix.shape[0]) or 
        ((candidate[0] - n) < 0) or
        ((candidate[0] + n) > matrix.shape[1])):
      # ignore it
      continue

    # defining limits 
    left = int(candidate[1] - n)
    right = int(candidate[1] + n)
    top = int(candidate[0] - n)
    bottom = int(candidate[0] + n)

    # getting patch / roi
    patchCandidate = matrix[top : bottom,
                            left : right]

    # starting the row values
    dictFeatures = {}
    dictFeatures = {'name': 'patch_' + str(index) + '_' + str(image.split(".")[0]),
                    #'label': patchGroundTruth(top, bottom, left, right, groundTruth),
                    'x': candidate[0],
                    'y': candidate[1]}

    # compute first order statistic features

    dictFos = functionFeatures(patchCandidate.astype(np.uint8))
    # saving it in the row values
    dictFeatures.update(dictFos)

      # Relative smoothness
    dictFeatures['relativeSmoothness'] = 1 - ( 1 / (1.0 + dictFeatures["Variance"]))
    
    # input image, distance in pixels, angles
    glcm = feature.greycomatrix(patchCandidate.astype(np.uint8), [ 1 ], [ 0 ])
    
    # glcm features    
    # properties
    dictFeatures['contrast'] = feature.greycoprops(glcm, 'contrast')[0][0]
    dictFeatures['dissimilarity'] = feature.greycoprops(glcm, 'dissimilarity')[0][0]
    dictFeatures['homogeneity'] = feature.greycoprops(glcm, 'homogeneity')[0][0]
    dictFeatures['energy'] = feature.greycoprops(glcm, 'energy')[0][0]
    dictFeatures['correlation'] = feature.greycoprops(glcm, 'correlation')[0][0]
    dictFeatures['ASM'] = feature.greycoprops(glcm, 'ASM')[0][0]

    dictFeatures['entropy'] = measure.shannon_entropy(glcm)
    fglcm = glcmFeatures(glcm[:,:,0,0])
    dictFeatures.update(fglcm)

    del glcm

    # add to the dataframe the features for this patch
    features = features.append(dictFeatures, ignore_index=True)

  # save in the csv
  print(folder)
  print(image)
  writeFeatures(features, flag, folder, image, '_fos_glcm')

  return features

## Additional Functions

In [ ]:
def imgDilation(matrix):
  kernel = np.ones((3,3), np.uint8)
  img_dilation = cv2.dilate(matrix, kernel, iterations=3)
  return img_dilation

In [ ]:
def runPipeline(imagePath, name, featuresOption, outputPath):

  # to save the features generated with the glcm
  features = pd.DataFrame(dtype=np.float64)

  imageName = imagePath.split('/')[-1]
  print(imageName)
  #upload images
  img = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
  imgCopy = copy.deepcopy(img)

  # pipelines
  if name == 'pipelineA':
    # preprocessing

    # dehazing
    preprocessed = cv2.cvtColor(imgCopy, cv2.COLOR_GRAY2BGR)

    preprocessed = deHazingDarkChannelPrior(preprocessed)

    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_dehazing.png'), preprocessed)

    # dilation
    preprocessedDil = imgDilation(preprocessed)

    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_dehazing_dilation.png'), preprocessedDil)

    # candidate extraction

    preprocessedDil = preprocessedDil.astype(np.float32)
    preprocessedDil = cv2.cvtColor(preprocessedDil, cv2.COLOR_BGR2GRAY)

    candidates = candidateExtractionDoG(preprocessedDil.astype(np.uint8), minSigma=0.0001, maxSigma=30, threshold=0.04)

  elif name == 'pipelineB':
    # preprocessing
    
    # clahe
    preprocessed = imgCLAHE(imgCopy)
    preprocessed = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_clahe.png'), preprocessed)

    # dehazing 
    preprocessed = deHazingDarkChannelPrior(preprocessed)
    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_clahe_dehazing.png'), preprocessed)

    # dilation
    preprocessedDil = imgDilation(preprocessed)
    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_clahe_dehazing_dilation.png'), preprocessedDil)
    # candidate extraction

    preprocessedDil = preprocessedDil.astype(np.float32)
    preprocessedDil = cv2.cvtColor(preprocessedDil, cv2.COLOR_BGR2GRAY)

    candidates = candidateExtractionDoG(preprocessedDil.astype(np.uint8), minSigma=0.005, maxSigma=50, threshold=0.04)


  elif name == 'pipelineD':
    # preprocessing

    # clahe
    preprocessed = imgCLAHE(imgCopy)
    preprocessed = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_clahe.png'), preprocessed)

    # dehazing 
    preprocessed = deHazingDarkChannelPrior(preprocessed)
    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_clahe_dehazing.png'), preprocessed)

    # dilation

    preprocessedDil = imgDilation(preprocessed)
    cv2.imwrite(os.path.join(outputPath, imageName.split('.')[0]+'_clahe_dehazing_dilation.png'), preprocessedDil)
    # candidate extraction

    preprocessedDil = preprocessedDil.astype(np.float32)
    preprocessedDil = cv2.cvtColor(preprocessedDil, cv2.COLOR_BGR2GRAY)

    candidates = candidateExtractionDoG(preprocessedDil.astype(np.uint8), minSigma=0.005, maxSigma=50, threshold=0.04)

  else:
    print('invalid pipeline')

  print('candidates detected, ', len(candidates))

  # feature extraction 

  copyPreprocessed = copy.deepcopy(preprocessed)
    
  copyPreprocessed = copyPreprocessed.astype(np.uint16)
  copyPreprocessed = cv2.cvtColor(copyPreprocessed, cv2.COLOR_BGR2GRAY)


  if featuresOption == 'GLCM':
    featuresExtractionGLCM(copyPreprocessed, candidates, features, imagePath, outputPath)
  elif featuresOption == 'HaarGLCM':
    featuresExtractionHaarGLCM(copyPreprocessed, candidates, features, imagePath, outputPath)
  elif featuresOption == 'LBPGLCM':
    featuresExtractionLBPGLCM(copyPreprocessed, candidates, features, imagePath, outputPath)


# Main

In [ ]:
DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')

runPipeline(os.path.join(DATA_DIR, 'images', '22580098_6200187f3f1ccc18_MG_L_ML_ANON.tif'),
            'pipelineD',
            'GLCM',
            os.path.join('/content',
                         'drive',
                         'MyDrive',
                         'test'))

22580098_6200187f3f1ccc18_MG_L_ML_ANON.tif
candidates detected,  1635


/content/drive/MyDrive/test
/content/drive/MyDrive/Image Processing and Analysis 2022/projects/Calcification Detection/dataset/images/22580098_6200187f3f1ccc18_MG_L_ML_ANON.tif
